Here we will see examples of using pre-built models to annotate text, and using NLP based features for classification.

First let's handle the imports and re-loading the data.

In [1]:
import scala.collection.immutable.TreeSet
import scala.sys.process._
import java.util

In [2]:
%AddJar file:/srv/toree/dist/toree-pip/NLP-on-Spark/annotate-1.0.jar
%AddJar file:/srv/toree/dist/toree-pip/NLP-on-Spark/opennlp-tools-1.6.0.jar
%AddJar file:/srv/toree/dist/toree-pip/NLP-on-Spark/models.jar

Starting download from file:/srv/toree/dist/toree-pip/NLP-on-Spark/annotate-1.0.jar
Finished download of annotate-1.0.jar
Starting download from file:/srv/toree/dist/toree-pip/NLP-on-Spark/opennlp-tools-1.6.0.jar
Finished download of opennlp-tools-1.6.0.jar
Starting download from file:/srv/toree/dist/toree-pip/NLP-on-Spark/models.jar
Finished download of models.jar


In [3]:
import nlp.spark.annotate.{ Annotation, Annotator, Document }

In [4]:
import org.apache.spark.ml.classification._
import org.apache.spark.ml.evaluation._
import org.apache.spark.ml.feature._
import org.apache.spark.ml.regression._
import org.apache.spark.ml.tuning._
import org.apache.spark.ml.Pipeline
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

In [5]:
val dataframe = sqlContext.read.parquet("annotated-1.parquet")

In [6]:
val annotated = dataframe.rdd.map {
  row =>
    val id = row.getString(0)
    val text = row.getString(1)
    val metadata = row.getMap[String, String](2).toMap
    val annotations = row.getSeq[Row](3).map {
      annoRow =>
        val aType = annoRow.getString(0)
        val begin = annoRow.getInt(1)
        val end = annoRow.getInt(2)
        val metadata = annoRow.getMap[String, String](3).toMap
        Annotation(aType, begin, end, metadata)
    }.to[TreeSet]
    Document(id, text, metadata, annotations)
}

In [7]:
annotated.count()

2744

In order to use the models from OpenNLP, we have them packaged into a jar file since OpenNLP's implementation of the models is not serializable.

For more information on how OpenNLP trains these models see  
http://repository.upenn.edu/cgi/viewcontent.cgi?article=1083&context=ircs_reports

In [8]:
"unzip -l models.jar" !

Archive:  models.jar
  Length      Date    Time    Name
---------  ---------- -----   ----
        0  2016-10-26 22:04   META-INF/
       68  2016-10-26 22:04   META-INF/MANIFEST.MF
    98533  2016-10-26 20:47   en-sent.bin
  5696197  2016-10-26 20:47   en-pos-maxent.bin
---------                     -------
  5794798                     4 files


In [9]:
import opennlp.tools.sentdetect.{ SentenceDetectorME, SentenceModel }
import opennlp.tools.postag.{ POSTaggerME, POSModel }

/**
  * Utility for loading models
  */
object ModelResource {
    val postaggerME = {
      val modelIS = getClass.getResourceAsStream("/en-pos-maxent.bin")
      new POSTaggerME(new POSModel(modelIS))
    }
    val sentDetectorME = {
      val modelIS = getClass.getResourceAsStream("/en-sent.bin")
      new SentenceDetectorME(new SentenceModel(modelIS))
    }
}

/**
  * This creates a part-of-speech tag annotations
  */
case class POSTagger() extends Annotator {
  private val annoType = "postag"
  
  override def annotate(
    text: String,
    metadata: Map[String, String] = Map(),
    annotations: TreeSet[Annotation] = TreeSet()): Seq[Annotation] = {
      annotations.iterator.filter(_.aType == "sentence").flatMap {
        sentence =>
          val tokens = annotations.range(
            Annotation("token", sentence.begin, sentence.begin),
            Annotation("token", sentence.end, sentence.end)
          ).toArray
          val postags = ModelResource.postaggerME.tag(tokens.map(t => text.substring(t.begin, t.end)))
          postags.zip(tokens).map {
            case (postag, token) =>
              Annotation(annoType, token.begin, token.end, Map("postag" -> postag))
          }
      }.toSeq
    }
}

/**
  * This creates sentence annotations
  */
case class SentAnnotator() extends Annotator {
  private val annoType = "sentence"
  
  override def annotate(
    text: String,
    metadata: Map[String, String] = Map(),
    annotations: TreeSet[Annotation] = TreeSet()): Seq[Annotation] = {
      ModelResource.sentDetectorME.sentPosDetect(text).map {
        span =>
          Annotation(annoType, span.getStart, span.getEnd)
      }.toSeq
    }
}

The part of speech of a word is a syntactic category words can be assigned. Let's look at an example.

In [10]:
val modelIS = getClass.getResourceAsStream("/en-pos-maxent.bin")
val tagger = new POSTaggerME(new POSModel(modelIS))

val tokens = "The North Wind and the Sun were having a dispute".split(" ")
tokens.zip(tagger.tag(tokens))

Array((The,DT), (North,NNP), (Wind,NNP), (and,CC), (the,DT), (Sun,NNP), (were,VBD), (having,VBG), (a,DT), (dispute,NN))

In the above example  
"DT" is a determinder, which are words like "the", "a", "some"  
"NNP" is a proper noun  
"CC" is a conjunction, like "and", "but", "or"  
"VBD" is a past tense verb  
"VBG" is a gerund  
"NN" is a noun

In [11]:
val sentAnnotator = SentAnnotator()
val postagger = POSTagger()

In [12]:
val annotated2 = annotated.map(sentAnnotator.apply).map(postagger.apply)

In [13]:
annotated2.count() // this action is triggering multiple previos lazy operations and may take a few moments

2744

Now let's train a model to detect whether a comment is actually a story or not.

First let's load the labels.

In [14]:
val labels = sqlContext.read.json("writingprompts-master/labeled_ids.json")

In [15]:
labels

[id: string, is_story: boolean]

These labels were gathered manually by one of the authors of this notebook.

In [16]:
labels.show(10)

+-------+--------+
|     id|is_story|
+-------+--------+
|ceeh6gj|    true|
|ceehmy9|    true|
|ceeikxs|    true|
|ceej0tr|   false|
|ceej1ki|   false|
|ceej8pt|    true|
|ceek7j9|   false|
|ceekv1c|   false|
|ceelhp7|    true|
|ceelsi2|   false|
+-------+--------+
only showing top 10 rows



Now we will join the datasets

In [17]:
val docType = 
  StructType(
    StructField("docName", StringType) ::
    StructField("text", StringType) ::
    StructField("metadata", MapType(StringType, StringType)) ::
    StructField("annotations", ArrayType(StructType(
      StructField("aType", StringType) ::
      StructField("begin", IntegerType) ::
      StructField("end", IntegerType) ::
      StructField("metadata", MapType(StringType, StringType)) :: Nil
    ))) :: Nil
  )

In [18]:
val dataframe2 = sqlContext.createDataFrame(
  annotated2.map {
    doc =>
      Row(
        doc.docName, 
        doc.text, 
        doc.metadata, 
        doc.annotations.map(a => Row(a.aType, a.begin, a.end, a.metadata)).toSeq
      )
  }, docType
)

In [19]:
val labeledDocDF = dataframe2.join(labels, col("docName") === col("id")).cache()

In [20]:
labeledDocDF.count() // this action is triggering multiple previos lazy operations and may take a few moments

698

In [21]:
labeledDocDF

[docName: string, text: string, metadata: map<string,string>, annotations: array<struct<aType:string,begin:int,end:int,metadata:map<string,string>>>, id: string, is_story: boolean]

Now let's define the features we will use to classify the stories and comments.

In order to train a model we will need to create numeric features since most ML algorithms are defined with numeric variables.

In [22]:
// Feature #1: By convention, only stories should be at the top level
val isTopLevel = when(
  expr("metadata['link_id']") === expr("metadata['parent_id']"), 1.0
).otherwise(0.0)

In [23]:
// Feature #2: length will likely be important, since comments are generally short
val textLength = length(col("text"))

In [24]:
// Feature #3: We will use the non stop words as features as well
// first let's create a UDF that takes the annotations and generates the sequence of non stop words
val getWordsUDF = sqlContext.udf.register(
  "getWords",
  (annotations: Seq[Row]) => {
    val stopwords = annotations.collect {
      case anno: Row if anno.getString(0) == "stopword" =>
        (anno.getInt(1), anno.getInt(2))
    }.toSet
    val words: Seq[String] = annotations.iterator.filter {
      case anno: Row => 
        anno.getString(0) == "ntoken" && !stopwords.contains(anno.getInt(1), anno.getInt(2))
    }.toSeq.sortBy(nt => (nt.getInt(1), -nt.getInt(2))).map(_.getMap[String, String](3)("ntoken"))
    words
  }: Seq[String]
)
val words = getWordsUDF(col("annotations"))

In [25]:
// Feature #4: Hypothetically, if a comment has genre specific words then that would seem to indicate 
// it is a story.
val countEntitiesUDF = sqlContext.udf.register(
  "countEntities",
  (annotations: Seq[Row], words: Seq[String]) => {
    val numEntities = annotations.count {
      case anno: Row =>
        anno.getString(0) == "entity"
    }.toDouble / words.size
    numEntities
  }: Double
)
val numEntities = countEntitiesUDF(col("annotations"), words)

In [26]:
// Feature #5: Sentence lengths may also be useful
val avgSentenceLengthsUDF = sqlContext.udf.register(
  "avgSentenceLengths",
  (annotations: Seq[Row]) => {
    val sentenceLengths = annotations.collect {
    case anno: Row if anno.getString(0) == "sentence" =>
      anno.getInt(2) - anno.getInt(1)
    }
    sentenceLengths.sum.toDouble / sentenceLengths.size
  }: Double
)
val avgSentenceLengths = avgSentenceLengthsUDF(col("annotations"))

In [27]:
// Feature #6: the score that comment gets may also be useful
val score = expr("metadata['score']").cast("double")

In [28]:
// the label we will train to match
val isStoryLabel = when(
  expr("is_story = true"), "story"
).otherwise("comment")

In [29]:
// Now we attach the columns
val labeledBoW = labeledDocDF.withColumn(
  "isTopLevel", isTopLevel
).withColumn(
  "words", words
).withColumn(
  "textLength", textLength
).withColumn(
  "numEntities", numEntities
).withColumn(
  "avgSentenceLengths", avgSentenceLengths
).withColumn(
  "score", score
).withColumn(
  "isStoryLabel", isStoryLabel
).cache()

In [30]:
labeledBoW.columns.zip(labeledBoW.first().toSeq).foreach(println)

(docName,ceeh6gj)
(text,It was 7:30 in the evening. A little bit after you would start a task, and a little bit before you would go to sleep. It is the worst time for a guest. I do not like guests. I prefer to be alone. That is why I moved to these apartments. Size was not important. Sound proofing and privacy were. This way I could sit in my silent home and not be bothered. But the incessant pounding on the door to my apartment would not relent. I heard #BADWORDS#uely claims that they were police officers.

I open the door as far as the chain will allow. I see two men in standard issue police uniforms. The taller man holds up his badge and starts talking at me. I do not like to be talked at. He does not seem to care that it was now 7:31, and not a good time for me to have guests.

"Mr. Faulk?", the tall officer asked.

"Yes", I replied. I do not like talking with people. I want to go back inside.

"Mr. Faulk, there has been a lot of reports of missing people around these buildings. I'

Now we will create our train-test split

In [31]:
val Array(trainDF, testDF) = labeledBoW.randomSplit(Array(0.75, 0.25), 123)

In [32]:
// We use a label indexer to convert the string into the vectorized version used by MLLib
var labelIndexer = new StringIndexer()
labelIndexer = labelIndexer.setInputCol("isStoryLabel")
labelIndexer = labelIndexer.setOutputCol("label")
val labelIndexModel = labelIndexer.fit(trainDF)
labelIndexModel.labels.toList

List(story, comment)

In [33]:
// This uses the "hashing trick" to turn the words into numeric features
// see for more information https://en.wikipedia.org/wiki/Feature_hashing
var hashingTF = new HashingTF()
hashingTF = hashingTF.setInputCol("words")
hashingTF = hashingTF.setOutputCol("wordfeatures")
hashingTF = hashingTF.setNumFeatures(25)

In [34]:
// This uses idf to downweight features that are common (as in tf.idf)
var idf = new IDF()
idf = idf.setInputCol("wordfeatures")
idf = idf.setOutputCol("tfidffeatures")

In [35]:
// This gathers all the features into a single vector
var vectorAssembler = new VectorAssembler()
vectorAssembler = vectorAssembler.setInputCols(Array(
  "isTopLevel", 
  "textLength",
  "avgSentenceLengths", 
  "numEntities",
  "tfidffeatures",
  "score"
))
vectorAssembler = vectorAssembler.setOutputCol("features")

In [36]:
// We will use a decistion tree to to classifiy the data
// for more information on decistion trees see https://en.wikipedia.org/wiki/Decision_tree_learning
var classifier = new DecisionTreeClassifier()
classifier = classifier.setLabelCol("label")
classifier = classifier.setFeaturesCol("features")
classifier = classifier.setSeed(123)
classifier = classifier.setMaxDepth(10)
classifier = classifier.setMinInstancesPerNode(10)
classifier = classifier.setImpurity("entropy")

In [37]:
// This converts the predicted labels from their vectorized form back into their string form
var labelConverter = new IndexToString()
labelConverter = labelConverter.setInputCol("prediction")
labelConverter = labelConverter.setOutputCol("isStoryPred")
labelConverter = labelConverter.setLabels(labelIndexModel.labels)

In [38]:
// Finally, we combine all of our steps into a pipeline
var pipeline = new Pipeline().setStages(Array(
  labelIndexer,
  hashingTF,
  idf,
  vectorAssembler,
  classifier,
  labelConverter
))

In [39]:
// We now fit our model to the training data
val model = pipeline.fit(trainDF)

In [40]:
// We run our model against (transform) the test data
val predictions = model.transform(testDF)

In [41]:
predictions.select("isStoryPred", "isStoryLabel").show(5)

+-----------+------------+
|isStoryPred|isStoryLabel|
+-----------+------------+
|    comment|       story|
|    comment|       story|
|    comment|     comment|
|    comment|     comment|
|    comment|     comment|
+-----------+------------+
only showing top 5 rows



In [42]:
// For evaluation we use the AUC (area under the curve)
// For more info on AUC see https://en.wikipedia.org/wiki/Receiver_operating_characteristic#Area_under_the_curve
// As a rule of thumb you can interpret values as:
// AUC < 0.5 worse than random
// 0.5 < AUC < 0.6 little better than random
// 0.6 < AUC < 0.7 the model has some value
// 0.7 < AUC < 0.9 a good model
// AUC > 0.9 a very good model
val evaluator = new BinaryClassificationEvaluator().setLabelCol("label").setRawPredictionCol("rawPrediction")
val auc = evaluator.evaluate(predictions)
println("AUC = " + auc)

AUC = 0.874023683547493


In [43]:
predictions.select("isStoryPred", "isStoryLabel").rdd.map {
  row =>
    s"truth: ${row.getString(1)}, prediction: ${row.getString(0)}"
}.groupBy(identity).mapValues(_.size).collect().foreach(println)

(truth: comment, prediction: comment,66)
(truth: story, prediction: story,76)
(truth: story, prediction: comment,22)
(truth: comment, prediction: story,15)


Before moving on, feel free to improve this model, because we will be using it to label the rest of our data.

In [44]:
val allData = dataframe2.withColumn(
  "isTopLevel", isTopLevel
).withColumn(
  "words", words
).withColumn(
  "textLength", textLength
).withColumn(
  "numEntities", numEntities
).withColumn(
  "avgSentenceLengths", avgSentenceLengths
).withColumn(
  "score", score
)

In [45]:
val allDataClassified = model.transform(allData)

In [46]:
val dataframe3 = allDataClassified.select(
  "docName",
  "text",
  "metadata",
  "annotations",
  "isTopLevel",
  "textLength",
  "avgSentenceLengths",
  "words",
  "numEntities",
  "score",
  "isStoryPred"
).cache()

In [47]:
dataframe3.count() // this action is triggering multiple previos lazy operations and may take a few moments

2744

In [48]:
dataframe3.filter("isStoryPred = 'story'").count()

1289

Let's save our featurized data for the next notebook.

In [49]:
if (new java.io.File("annotated-2.parquet").exists) {
  "rm -rf annotated-2.parquet" !
}

In [50]:
dataframe3.write.parquet("annotated-2.parquet")

Now that we can separate the stories from the comments, let's try and build a model to predict a good scoring story. Normally, this is approached through regression, but the distribution of votes in this data set has very low variance. This can normally be worked around if one has a large dataset, however, this dataset is small for demonstration purposes. Later notebooks don't depend on the output of this data, so feel free to build a model using regression or classification.

In [51]:
val storiesDF = dataframe3.filter(col("isStoryPred") === "story")

In [52]:
storiesDF.count()

1289

In [53]:
val scores = storiesDF.select("score").rdd.map(_.getDouble(0)).collect()
scores.groupBy(identity).mapValues(_.size).toSeq.sortBy(_._1).foreach {
  case (score, count) =>
    println(f"$score%- 10.3f ${"*" * scala.math.round(count / 10.0).toInt}%-10s")
}

-5.000               
-2.000               
-1.000               
 0.000     *         
 1.000     ************************************
 2.000     *****************************
 3.000     ******************
 4.000     ****      
 5.000     *****     
 6.000     ******    
 7.000     ****      
 8.000     ***       
 9.000     ***       
 10.000    *         
 11.000    *         
 12.000    *         
 13.000    *         
 14.000    *         
 15.000              
 16.000    *         
 17.000    *         
 18.000              
 19.000    *         
 20.000    *         
 21.000    *         
 22.000              
 24.000    *         
 25.000              
 26.000              
 27.000              
 28.000              
 29.000              
 30.000    *         
 31.000              
 32.000              
 36.000              
 37.000              
 38.000              
 39.000              
 40.000              
 41.000              
 42.000              
 44.000              
 

Here we create the binary target in case you decide to use a classifier.

In [54]:
val storiesWithPopDF = storiesDF.withColumn(
  "popularity",
  when(
    col("score") <= 2.0, "not-popular"
  ).otherwise("popular")
)

We will save this for future use.

In [55]:
if (new java.io.File("stories.parquet").exists) {
  "rm -rf stories.parquet" !
}

In [56]:
storiesWithPopDF.write.parquet("stories.parquet")

In [57]:
val Array(trainScoreDF, testScoreDF) = storiesWithPopDF.randomSplit(Array(0.75, 0.25), 123)

### Exercise 2. Create a model for predicting the score

### Solution

#### Regression Solution

In [58]:
var vectorAssemblerScore = new VectorAssembler()
vectorAssemblerScore = vectorAssemblerScore.setInputCols(Array(
  "textLength",
  "avgSentenceLengths", 
  "numEntities",
  "tfidffeatures"
))
vectorAssemblerScore = vectorAssemblerScore.setOutputCol("features")

var regressor = new DecisionTreeRegressor()
regressor = regressor.setFeaturesCol("features")
regressor = regressor.setLabelCol("score")
regressor = regressor.setPredictionCol("scorePred")
regressor = regressor.setSeed(123)
regressor = regressor.setMaxDepth(10)
regressor = regressor.setMinInstancesPerNode(20)

var pipelineScore = new Pipeline().setStages(Array(
  hashingTF,
  idf,
  vectorAssemblerScore,
  regressor
))

In [59]:
val modelScore = pipelineScore.fit(trainScoreDF)

In [60]:
val predictionsScore = modelScore.transform(testScoreDF)

In [61]:
predictionsScore.select("score", "scorePred").show(10)

+-----+------------------+
|score|         scorePred|
+-----+------------------+
|  2.0|             28.85|
|  3.0|1.6666666666666667|
|  6.0|1.6666666666666667|
|  2.0| 3.111111111111111|
|  1.0| 5.552631578947368|
|  4.0| 70.94285714285714|
| 79.0|             28.85|
|  6.0|2.0576923076923075|
|  1.0| 4.940298507462686|
|  8.0|10.766666666666667|
+-----+------------------+
only showing top 10 rows



In [62]:
val evaluator = new RegressionEvaluator().setLabelCol("score").setPredictionCol("scorePred")
val rmse = evaluator.evaluate(predictionsScore)
println("RMSE = " + rmse)
val mae = evaluator.setMetricName("mae").evaluate(predictionsScore)
println("MAE = " + mae)
val r2 = evaluator.setMetricName("r2").evaluate(predictionsScore)
println("R2 = " + r2)

RMSE = 71.11339166196683
MAE = 17.825087598699607
R2 = 0.003031565345640641


#### Classification solution

In [63]:
var labelIndexerScore = new StringIndexer()
labelIndexerScore = labelIndexerScore.setInputCol("popularity")
labelIndexerScore = labelIndexerScore.setOutputCol("popularityLabel")
val labelIndexScoreModel = labelIndexerScore.fit(trainScoreDF)
labelIndexScoreModel.labels.toList

List(not-popular, popular)

In [64]:
var vectorAssemblerScore = new VectorAssembler()
vectorAssemblerScore = vectorAssemblerScore.setInputCols(Array(
  "textLength",
  "avgSentenceLengths", 
  "numEntities",
  "tfidffeatures"
))
vectorAssemblerScore = vectorAssemblerScore.setOutputCol("features")

var classifierScore = new DecisionTreeClassifier()
classifierScore = classifierScore.setFeaturesCol("features")
classifierScore = classifierScore.setLabelCol("popularityLabel")

var labelConverterScore = new IndexToString()
labelConverterScore = labelConverterScore.setInputCol("prediction")
labelConverterScore = labelConverterScore.setOutputCol("popularityPred")
labelConverterScore = labelConverterScore.setLabels(labelIndexScoreModel.labels)

var pipelineScore = new Pipeline().setStages(Array(
  labelIndexerScore,
  hashingTF,
  idf,
  vectorAssemblerScore,
  classifierScore,
  labelConverterScore
))

In [65]:
val modelScore = pipelineScore.fit(trainScoreDF)

In [66]:
val predictionsScore = modelScore.transform(testScoreDF)

In [67]:
predictionsScore

[docName: string, text: string, metadata: map<string,string>, annotations: array<struct<aType:string,begin:int,end:int,metadata:map<string,string>>>, isTopLevel: double, textLength: int, avgSentenceLengths: double, words: array<string>, numEntities: double, score: double, isStoryPred: string, popularity: string, popularityLabel: double, wordfeatures: vector, tfidffeatures: vector, features: vector, rawPrediction: vector, probability: vector, prediction: double, popularityPred: string]

In [68]:
var evaluatorScore = new BinaryClassificationEvaluator()
evaluatorScore = evaluatorScore.setLabelCol("popularityLabel")
evaluatorScore = evaluatorScore.setRawPredictionCol("rawPrediction")
val auc = evaluatorScore.evaluate(predictionsScore)
println("AUC = " + auc)

AUC = 0.5297175141242938


In [69]:
predictionsScore.select("popularityPred", "popularity").rdd.map {
  row =>
    s"truth: ${row.getString(1)}, prediction: ${row.getString(0)}"
}.groupBy(identity).mapValues(_.size).collect().foreach(println)

(truth: not-popular, prediction: not-popular,103)
(truth: not-popular, prediction: popular,74)
(truth: popular, prediction: not-popular,81)
(truth: popular, prediction: popular,69)


### end of exercise 